In [1]:
import sys
import csv
import pickle

We have a CSV file with one column taken from a database. This column contains the type of an item, such as a document or news item. Lets take a look at a small sample:

In [2]:
with open('itemtypes.csv', newline='') as csvfile:
    data = [i[0] for i in list(csv.reader(csvfile))][1:]

print(f'{len(data)} items in CSV.\n\nSample of 10 items:\n\n', data[:10])

500 items in CSV.

Sample of 10 items:

 ['agenda-item', 'news-item', 'vacancy', 'vacancy', 'vacancy', 'vacancy', 'news-item', 'vacancy', 'forum-topic', 'vacancy']


We can encode all the different types in a dictionary so we can represent it by an integer instead of the full string.

In [3]:
def get_dictionaries(data):
    encode_dictionary = { x: i for i, x in enumerate(set(data)) }
    decode_dictionary = { i: x for i, x in enumerate(set(data)) }
    
    return encode_dictionary, decode_dictionary

In [4]:
encode_dictionary, decode_dictionary = get_dictionaries(data)

print(encode_dictionary)
print(decode_dictionary)

{'document': 0, 'vacancy': 1, 'bulletin-item': 2, 'expert-panel-question': 3, 'agenda-item': 4, 'page': 5, 'dossier': 6, 'generic-item': 7, 'forum-topic': 8, 'news-item': 9, 'venue': 10}
{0: 'document', 1: 'vacancy', 2: 'bulletin-item', 3: 'expert-panel-question', 4: 'agenda-item', 5: 'page', 6: 'dossier', 7: 'generic-item', 8: 'forum-topic', 9: 'news-item', 10: 'venue'}


In [5]:
encoded_data = [encode_dictionary.get(item) for item in data]
print(encoded_data[:100])

[4, 9, 1, 1, 1, 1, 9, 1, 8, 1, 1, 9, 9, 1, 1, 1, 1, 8, 8, 0, 8, 9, 1, 9, 8, 9, 7, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 9, 1, 1, 1, 1, 9, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 2, 1, 1, 9, 1, 1, 1, 1, 1, 1, 1, 1, 1, 10, 9, 9, 3, 1, 9, 0, 8, 9, 10, 1, 1, 1, 0, 8, 1, 9, 1, 0, 1, 1, 1, 1, 1, 9, 9, 1, 1, 1, 1, 1, 0, 0]


As you can see, there are many repetitions of values, such as 8, 8, 8, 8. We can use _run-length encoding_ to save more space. We could encode 8, 8, 8, 8 as (8, 4), leaving us with just 2 integers: one for the actual value, and one for the run length. 

What if the rows are ordered such that there are no long runs of the same value? In case of a real database, where each row has multiple columns you'll have to find the most compact order. Computing the best ordering is an NP-hard problem. Most database systems therefore use a set of heuristics to give a good compaction but runs in a short amount of time.

In this case, for demonstration purposes, we'll just use the existing array as it already has many repetitive item types and use run-length encoding. Lets take a look at the result:

In [6]:
def run_length_encode(data):
    result = []
    
    count = 1
    last_item = None
    length = len(data)

    for i, item in enumerate(data):    
        if last_item is item:
            count += 1
            if i == (length - 1):
                result.append((last_item, count))
            
        elif last_item != None:
            result.append((last_item, count))
            count = 1
            
        last_item = item
            
    return result

In [7]:
print(len(run_length_encode(encoded_data)))
run_length_encode(encoded_data)[:10]

279


[(4, 1),
 (9, 1),
 (1, 4),
 (9, 1),
 (1, 1),
 (8, 1),
 (1, 2),
 (9, 2),
 (1, 4),
 (8, 2)]

We went from 500 items to just 11 items in the list!

In [12]:
def describe(data):
    encode_dict, _ = get_dictionaries(data)
    print('Encoding dictionary:\n', encode_dict)

    encoded_data = [encode_dict.get(item) for item in data]
    print('\nData:\n', data[:5])
    print('\nDictionairy encoded:\n', encoded_data[:5])
    
    full_encoded_data = run_length_encode(encoded_data)
    print('\nDictionairy + Run-length Encoded:\n', full_encoded_data[:5])
    
    original_size = sys.getsizeof(pickle.dumps(data))
    final_size = sys.getsizeof(pickle.dumps(full_encoded_data))

    print(
        f'\nOriginal: {original_size}\n' \
        f'Encoded: {sys.getsizeof(pickle.dumps(encoded_data))}\n' \
        f'Dictionairy + Run-length Encoded: {final_size}\n' \
        f'That is a {100 - (final_size * 100 // original_size)}% reduction in size.'
    )

In [13]:
describe(data)

Encoding dictionary:
 {'document': 0, 'vacancy': 1, 'bulletin-item': 2, 'expert-panel-question': 3, 'agenda-item': 4, 'page': 5, 'dossier': 6, 'generic-item': 7, 'forum-topic': 8, 'news-item': 9, 'venue': 10}

Data:
 ['agenda-item', 'news-item', 'vacancy', 'vacancy', 'vacancy']

Dictionairy encoded:
 [4, 9, 1, 1, 1]

Dictionairy + Run-length Encoded:
 [(4, 1), (9, 1), (1, 4), (9, 1), (1, 1)]

Original: 8429
Encoded: 1041
Dictionairy + Run-length Encoded: 2066
That is a 76% reduction in size.


This is a 76% reduction in size! All this makes indexing a whole lot easier as well. When scanning all of the rows, you can just index into the lookup table.

Notice how the dictionary + run-length encoding is actually longer. This is because there is not enough repetitiveness in the dataset to make this efficient. If we would sort the dataset we would get an even better compaction (however this is unrealistic when you have more than 1 column).

In [15]:
describe(sorted(data))

Encoding dictionary:
 {'forum-topic': 0, 'document': 1, 'bulletin-item': 2, 'vacancy': 3, 'agenda-item': 4, 'page': 5, 'dossier': 6, 'generic-item': 7, 'expert-panel-question': 8, 'news-item': 9, 'venue': 10}

Data:
 ['agenda-item', 'agenda-item', 'agenda-item', 'agenda-item', 'agenda-item']

Dictionairy encoded:
 [4, 4, 4, 4, 4]

Dictionairy + Run-length Encoded:
 [(4, 35), (2, 2), (1, 101), (6, 3), (8, 1)]

Original: 8429
Encoded: 1041
Dictionairy + Run-length Encoded: 118
That is a 99% reduction in size.
